In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import requests, zipfile, io #웹데이터 가져오기
from bs4 import BeautifulSoup #데이터 추출하기

key = "569e4ca329b73228c7d8aa872f8c629ad715712"
url = f'http://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={key}'
r = requests.get(url)

#바이너리 압축파일을 불러온 이후, 압축을 해제한 xml 파일 저장하기
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall('./data')

#필요한 정보 추출 & 함수로 만들어서 유용하게 활용하기
def get_companynumber(inputcode):
    xml = './data/CORPCODE.xml'
    read_xml = open(xml, "r", encoding='utf-8').read()
    soup = BeautifulSoup(read_xml, 'lxml')
    
    items = soup.find_all('list')
    for item in items :
        stockcode = item.find('stock_code').text
        if stockcode == str(inputcode):
            company_name = item.find('corp_name').text
            company_number = item.find('corp_code').text
            print(f'company number: {company_number} \ncompany name: {company_name}')
    return None

In [3]:
import requests 
import pandas as pd 
import io 
import zipfile 
import xml.etree.ElementTree as et 
import json

In [5]:
def get_corpcode(crtfc_key): 
    params = {'crtfc_key':crtfc_key} 
    url = "https://opendart.fss.or.kr/api/corpCode.xml" 
    items_en = ["corp_code", "corp_name", "stock_code", "modify_date"] 
    
    ## Column명을 한글로 하려면 아래부분 쌍따옴표안을 수정하면 됨 
    items_kr = ["corp_code", "corp_name", "stock_code", "modify_date"] 
    results = requests.get(url,params=params) 
    uzfile = zipfile.ZipFile(io.BytesIO(results.content)) 
    final = uzfile.open(uzfile.namelist()[0]) 
    root = et.fromstring(final.read().decode('utf-8')) 
    data = [] 
    for child in root: 
        if len(child.find('stock_code').text.strip()) > 1: 
            data.append([]) 
            for item in items_en: 
                data[-1].append(child.find(item).text) 
    df = pd.DataFrame(data, columns = items_kr) 
    ##print(df) 
    return df

In [12]:
crtfc_key = "-" 
code_df = get_corpcode(crtfc_key)
code_df.head()

,corp_code,corp_name,stock_code,modify_date
0,00260985,한빛네트,036720,20170630
1,00264529,엔플렉스,040130,20170630
2,00358545,동서정보기술,055000,20170630
3,00231567,애드모바일,032600,20170630
4,00247939,씨모스,037600,20170630


In [20]:
company_name_df = pd.read_csv('/Users/kimhansaem/workspace/11.DataAnalysis/01.데이터분석/data/반도체 기업 목록.txt', sep='\s+', header=None,
                                names=['column_index', 'column_name'])

In [21]:
company_names = company_name_df['column_index'].values.tolist()
print(company_names[:20])

['큐에스아이', '예스티', '미코', '신성이엔지', '코세스', '디엔에프', '로체시스템즈', '휘닉스소재', '주성엔지니어링', '라온테크', '알에프세미', '지오엘리먼트', '넥스틴', '타이거일렉', '나인테크', '뉴파워프라즈마', '젬백스', '다믈멀티미디어', '메카로', '램테크놀러지']


In [22]:
df_sample = code_df[code_df['corp_name'].isin(company_names)]
df_sample.head(3)

,corp_code,corp_name,stock_code,modify_date
611,00686068,신성이엔지,104110,20171208
784,00411905,테라셈,182690,20220114
787,00625942,아이원스,114810,20220117


In [23]:
df_sample.nunique()

corp_code      127
corp_name      126
stock_code     127
modify_date     44
dtype: int64

In [19]:
df_sample.to_excel('고유번호.xlsx')

In [14]:
df = pd.read_csv('/Users/kimhansaem/workspace/11.DataAnalysis/01.데이터분석/data/2017_반기보고서_01_재무상태표_연결_20220212.txt', sep = '\t', encoding='euc-kr')
column_names = ['종목코드', '회사명', '업종', '업종명', '항목명', '당기 반기말']
df = df[column_names]
df.head()

,종목코드,회사명,업종,업종명,항목명,당기 반기말
0,[060310],3S,292,특수 목적용 기계 제조업,자산 [abstract],NaN
1,[060310],3S,292,특수 목적용 기계 제조업,유동자산,"12,963,850,356"
2,[060310],3S,292,특수 목적용 기계 제조업,현금및현금성자산,"2,686,944,114"
3,[060310],3S,292,특수 목적용 기계 제조업,단기금융상품,"984,355,188"
4,[060310],3S,292,특수 목적용 기계 제조업,매출채권및기타채권,"5,908,239,202"


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67338 entries, 0 to 67337
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   종목코드    67338 non-null  object
 1   회사명     67338 non-null  object
 2   업종      67338 non-null  int64 
 3   업종명     67338 non-null  object
 4   항목명     67338 non-null  object
 5   당기 반기말  61818 non-null  object
dtypes: int64(1), object(5)
memory usage: 3.1+ MB


In [16]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61818 entries, 1 to 67337
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   종목코드    61818 non-null  object
 1   회사명     61818 non-null  object
 2   업종      61818 non-null  int64 
 3   업종명     61818 non-null  object
 4   항목명     61818 non-null  object
 5   당기 반기말  61818 non-null  object
dtypes: int64(1), object(5)
memory usage: 3.3+ MB


In [17]:
df['종목코드'] = df['종목코드'].apply(lambda x : x.replace('[', ''))
df['종목코드'] = df['종목코드'].apply(lambda x : str(x.replace(']', '')))
df['항목명'] = df['항목명'].apply(lambda x : x.strip())
df['항목명'] = df['항목명'].str.replace('[^가-힣 ]', '')
df['항목명'] = df['항목명'].str.replace('이익잉여금결손금', '이익잉여금')
df['항목명'] = df['항목명'].str.replace('당기말', '자본총계')
df['당기 반기말'] = df['당기 반기말'].apply(lambda x : int(x.replace(',', '')))
df['항목명'] = df['항목명'].str.replace('결손금', '이익잉여금')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61818 entries, 1 to 67337
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   종목코드    61818 non-null  object
 1   회사명     61818 non-null  object
 2   업종      61818 non-null  int64 
 3   업종명     61818 non-null  object
 4   항목명     61818 non-null  object
 5   당기 반기말  61818 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 3.3+ MB


In [18]:
df.head(10)

,종목코드,회사명,업종,업종명,항목명,당기 반기말
1,060310,3S,292,특수 목적용 기계 제조업,유동자산,12963850356
2,060310,3S,292,특수 목적용 기계 제조업,현금및현금성자산,2686944114
3,060310,3S,292,특수 목적용 기계 제조업,단기금융상품,984355188
4,060310,3S,292,특수 목적용 기계 제조업,매출채권및기타채권,5908239202
5,060310,3S,292,특수 목적용 기계 제조업,미청구공사,1771995945
6,060310,3S,292,특수 목적용 기계 제조업,재고자산,1053989746
7,060310,3S,292,특수 목적용 기계 제조업,당기법인세자산,210648240
8,060310,3S,292,특수 목적용 기계 제조업,기타유동자산,242798161
9,060310,3S,292,특수 목적용 기계 제조업,파생금융상품,104879760
10,060310,3S,292,특수 목적용 기계 제조업,비유동자산,37061149131


In [19]:
df['종목코드'].nunique()

1487

In [21]:
company_name_df2 = pd.read_csv('/Users/kimhansaem/workspace/11.DataAnalysis/01.데이터분석/data/시총100.txt', sep='\s+', header=None,
                                names=['column_index', 'column_name'])

In [22]:
company_names2 = company_name_df2['column_index'].values.tolist()
print(company_names2[:20])

['삼성전자', 'LG에너지솔루션', 'SK하이닉스', '삼성전자우', 'NAVER', '삼성바이오로직스', '카카오', 'LG화학', '삼성SDI', '현대차', '기아', 'POSCO', 'KB금융', '카카오뱅크', '셀트리온', '현대모비스', '삼성물산', 'LG전자', '카카오페이', '신한지주']
